In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [2]:
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

In [21]:
print(vars(train_data[1]))

{'text': ['of', 'the', 'films', 'of', 'the', 'young', 'republic', 'few', 'in', 'number', 'as', 'they', 'are', 'The', 'Buccaneer', '(', '1958)stands', 'out', 'as', 'a', 'finely', 'crafted', 'film', '.', 'Charleton', 'Heston', 'excels', 'in', 'his', 'portrayal', 'of', 'Old', 'Hickory', "'s", 'defence', 'of', 'New', 'Orleans', 'with', 'a', 'thrown', 'together', 'force', 'of', 'militia', ',', 'regulars', 'and', 'pirates', 'promised', 'a', 'reprieve.<br', '/><br', '/>after', 'Christmas', '1814', 'peninsula', 'veterans', 'led', 'by', 'sir', 'edward', 'packenham', ',', 'the', 'duke', 'of', 'wellington', "'s", 'brother', 'in', 'law', 'bore', 'down', 'on', 'the', 'city', 'of', 'new', 'orleans', '.', 'andy', 'jackson', 'had', 'a', 'day', 'to', 'draw', 'together', 'a', 'scratch', 'force', 'to', 'defend', 'the', 'city', 'behind', 'bales', 'of', 'hay.<br', '/><br', '/>Charlton', 'Heston', 'projects', 'Jackson', "'s", 'terrifying', 'presence', 'and', 'awe', 'inspiring', 'power', 'of', 'command', '.'

In [3]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device)

In [4]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden.squeeze(0))

In [5]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [6]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [7]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.2837, -0.6263, -0.4435,  ...,  0.4368, -0.8261, -0.1570],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1468,  0.3804,  0.4931,  ...,  1.8147,  0.1932,  0.6185],
        [ 0.2095,  0.2334,  0.4693,  ..., -0.5408, -1.0424,  0.1241],
        [ 0.9397, -0.6000, -0.4878,  ...,  0.1300,  0.5543,  0.6509]])

In [8]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [9]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [10]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [11]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [12]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

In [14]:
for batch in train_iterator:
    print(batch.text)

tensor([[   11,  9879,    14,  ...,  7895,    66,    11],
        [   76,    65,    25,  ...,     3,    23,   172],
        [  433,     8, 11486,  ...,   366,    19,     5],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[  158, 24631,   507,  ..., 11557,    25,    11],
        [ 5627,   844,    36,  ..., 20816,    24,    75],
        [    3,  8584,  1196,  ...,   655,    15,  3413],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[  11,   11,   66,  ...,   25, 5648,   66],
        [1589,  265, 3255,  ...,   73,    9,   19],
        [  16,    5,  252,  ...,   17,    5,  408],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
   

tensor([[ 11,  11,  66,  ...,  11,  11,  66],
        [ 74, 597,   9,  ..., 267, 267,  23],
        [546,  83,  38,  ...,  94,  36,  19],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])
tensor([[4082,   11, 1560,  ...,   54,  491,   66],
        [ 152,   75,  103,  ...,   51,    3,  534],
        [  36,  360, 5831,  ...,   92,   19, 1621],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[ 2066,    11,    11,  ...,   375,  2220, 17164],
        [    3,   546,   532,  ...,   175,     3,  5053],
        [  162,    12,    16,  ...,    44,    11,     9],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[ 548,   54,  

tensor([[ 147,   66,   66,  ...,   11, 6099,   25],
        [ 161,   19,   24,  ..., 2134,   19,   23],
        [   0,    2,    9,  ...,   64, 7306,    9],
        ...,
        [   1, 1632,    1,  ...,    1,    1,    1],
        [   1,    4,    1,  ...,    1,    1,    1],
        [   1, 2439,    1,  ...,    1,    1,    1]])
tensor([[   66, 16950,    25,  ...,    11,    25,   393],
        [    9,    29, 17994,  ...,  6572,    23,  3221],
        [    5,  5999, 18142,  ...,   497,  3302,    90],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[2530,   66,   66,  ...,   11,  322, 1538],
        [4395,   24,    9,  ...,  253,   11,  199],
        [   6,    9,   28,  ...,   47,  239, 6183],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1, 

tensor([[  25,   11,   66,  ...,   66, 8741,   25],
        [ 106,  518,    9,  ...,    9, 8098, 4389],
        [ 173,   28,   39,  ...,   38,   51,  270],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[18144,     0,   263,  ...,    11,    25,    66],
        [    9,   501,     9,  ...,    19,    74,    24],
        [   10,     6,    16,  ...,    38,    24,     9],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[ 5257,   580,  3351,  ...,   464,    66,    66],
        [ 3945,   284,     2,  ..., 14591,   145,  1198],
        [   90,    78,  2217,  ...,  7112,   663,    17],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [

tensor([[   11,    66,   359,  ..., 10199,  2020,   147],
        [   34,    76,     7,  ..., 20798,     6,   426],
        [  498,     9,     2,  ...,     9,    24,   739],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[   0,   66,   25,  ...,  460,   11,  155],
        [1285,   24,   63,  ...,   10,  144,    9],
        [  53,   19,  106,  ...,    2,  416,    5],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[ 147,  504, 1252,  ..., 1594,   11,  961],
        [5221,   28,  103,  ...,    5,   62, 6748],
        [   6,  192,   23,  ..., 2846,   47,    7],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1, 

KeyboardInterrupt: 

In [20]:
import torchtext
torchtext.__version__

'0.3.1'